# Amazing RL Agent

In [1]:
import gymnasium as gym
import torch
import numpy as np
import importlib

def moving_average(x, w):
    return np.convolve(x, np.ones(w), 'same') / w

c:\Users\Guard\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Test pytorch is working

x = torch.rand(size=(1000,))
x = x + 1

# If you have a nvidia gpu with torch installed correctly
# you can move the tensors to the gpu for some extra
# speed, not necessary although.
DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"
print(f"Using {DEVICE}")
# When creating a new tensor make sure you move it to a
# device using .to(DEVICE), such as:
x = x.to(DEVICE)
# The tensor in x has been moved to the device


Using cuda:0


In [11]:
# Import agent
import agents
import ddpg

# Reload module as jupyter notebooks wont update to any changes youve made
importlib.reload(
    agents,
)
importlib.reload(
    ddpg,
)
#agent = agents.Agent()


RENDER_AGENT = None

#agent = agents.TD3(24, 4, 1, noise=0.25, copy_step=3, device=DEVICE)
agent = ddpg.DDPG(24, 4, 1, batch_size=32, device=DEVICE)
scores = []
import matplotlib.pyplot as plt


In [12]:
try:
    env.close()
except:
    pass

env = gym.make("BipedalWalker-v3", hardcore=False, render_mode=RENDER_AGENT)
plt.figure(figsize=(1600/96, 800/96), dpi=96)
observation, info = env.reset(seed=27)
steps_per_episode = 1600 # T
for episode in range(500): # M
    rewards = 0
    step = 0
    while(step < steps_per_episode):
        action = agent.choose_action(observation, env.action_space).cpu()  # this is where you would insert your policy
        
        action = np.array(action)
        observation, reward, terminated, truncated, info = env.step(action)
        
        rewards += reward

        reward_agent = 100 * reward if reward > 0 else reward
        
        agent.update(observation, reward_agent, terminated, truncated)
        if terminated or truncated:
            observation, info = env.reset()

        step += 1
    scores.append(rewards)
    if episode % 1 == 0:
        print(f"Episode: {episode}, Reward: {rewards}, Memory Size: {agent.memory.size}")
    """
    m = max(scores) / steps_per_episode
    plt.clf()
    plt.title(f"Best: {m}")
    plt.plot(np.array(scores) / steps_per_episode)
    plt.axhline(y=0, color="grey", linestyle="-")
    plt.axhline(y=m, color="yellow", linestyle="-")
    plt.axhline(y=300 / steps_per_episode, color="red")
    plt.savefig("training.png")
"""

    ma = max(scores) 
    mi = min(scores) 
    plt.clf()
    plt.figure(figsize=(1600/96, 800/96), dpi=96)
    plt.title(f"Best: {ma}")
    plt.ylim((mi, ma))
    plt.xlim((0, len(scores)))

    plt.plot(np.array(scores), color="lightblue")
    plt.plot(moving_average(np.array(scores), 64)[:-32], color="blue") 
    plt.axhline(y=0, color="black", linestyle="-")
    plt.axhline(y=300, color="red", linestyle="-")
    plt.savefig("training.png")
    if RENDER_AGENT != None: 
        env.render()
    
    observation, info = env.reset()
env.close()


Episode: 0, Reward: -187.49583554725797, Memory Size: 1600


<ipython-input-12-b0c1fdba6034>:47: UserWarning: Attempting to set identical bottom == top == -187.49583554725797 results in singular transformations; automatically expanding.
  plt.ylim((mi, ma))


Episode: 1, Reward: -82.0268382717042, Memory Size: 3200
Episode: 2, Reward: -398.5422989419083, Memory Size: 4800
Episode: 3, Reward: -85.08709299097629, Memory Size: 6400
Episode: 4, Reward: -183.19026604667908, Memory Size: 8000
Episode: 5, Reward: -954.5583588715284, Memory Size: 9600
Episode: 6, Reward: -952.9704558075687, Memory Size: 11200
Episode: 7, Reward: -1404.4507266128887, Memory Size: 12800
Episode: 8, Reward: -2315.426510251485, Memory Size: 14400
Episode: 9, Reward: -1635.6896739610124, Memory Size: 16000
Episode: 10, Reward: -883.1650240992524, Memory Size: 17600


In [72]:
torch.save(agent.actor.state_dict(), "modded_learning_ddpg_actor.pth")
torch.save(agent.critic.state_dict(), "modded_learning_ddpg_critic.pth")

np.savetxt("scores.csv", scores, delimiter=",")
